## Домашнее задание к лекции "Основы веб-скрапинга и работы с API"
### Задание 1.
#### Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

#### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
keywords = ['python', 'парсинг']

In [4]:
import pandas as pd

In [5]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [6]:
posts = soup.find_all('article', class_='post post_preview')


In [7]:
df_post = pd.DataFrame()
for post in posts:
    post_id = post.parent.attrs.get('id')
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    post_lower = post.text.lower()
    if any([post_lower in desired for desired in keywords]):
        date_element = post.find('span', class_ = 'post__time')
        title_element = post.find('a', class_='post__title_link')
        text_element = post.find('div', class_ = 'post__text post__text-html post__text_v1')
        row = {'date': date_element.text, 'title': title_element.text, 'href': title_element.attrs.get('href'), 'text': text_element.text}
        df_post = pd.concat([df_post, pd.DataFrame([row])])
        break
    else:
        page = post.find('a', class_='btn btn_x-large btn_outline_blue post__habracut-btn')
        textpost = requests.get(page.attrs.get('href'))
        soup_text = BeautifulSoup(textpost.text, 'html.parser')
        text_posts = soup_text.find_all('div', class_='post__wrapper')
        for text in text_posts:
            text_lower = text.text.lower()
            for keywords_el in keywords:
                if keywords_el in text_lower:
                    date_element = post.find('span', class_ = 'post__time')
                    title_element = post.find('a', class_='post__title_link')
                    text_element = post.find('div', class_ = 'post__text post__text-html post__text_v1')
                    row = {'date': date_element.text, 'title': title_element.text, 'href': title_element.attrs.get('href'), 'text': text_element.text}
                    df_post = pd.concat([df_post, pd.DataFrame([row])])
                    break
            break

In [8]:
from datetime import datetime
now = datetime.now()
date_ = datetime.strftime(now, '%d.%m.%Y')

In [9]:
def add_date(date):
    if 'сегодня ' in date:
        return date_

In [10]:
df_post['date2'] = df_post['date'].apply(add_date)
df_post['date'] = df_post['date'].str.replace('сегодня в', ' ')
df_post['date'] = df_post['date2'] + df_post['date']
del df_post['date2']
df_post

,date,title,href,text
0,31.08.2020 16:16,Пошаговое руководство по созданию торгового бо...,https://habr.com/ru/company/ruvds/blog/517234/,"\n У меня нет квалификации, позволяющей д..."
0,31.08.2020 15:43,Обзор OpenStack Neutron PTG июнь 2020,https://habr.com/ru/company/huawei/blog/517248/,\n\n\r\nPTG (Project Team Gathering) – это мер...
0,31.08.2020 15:42,Новое поколение разработчиков. Чем они отличаю...,https://habr.com/ru/company/sberbank/blog/516698/,"\n\n\r\nСуществует мнение, что незаметно в IT ..."


### Задание 2.
#### Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

#### Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [94]:
email = ['alekc@donet.ru', 'a.yarmullin@gmail.com']
email_d ={}
postname = list()
date = list()
title = list()
description = list()
dict_={'postname':postname, 'date': date, 'title': title, 'description': description}
import time
df_email = pd.DataFrame()
for el in email:
    email_d['email'] = el
    avast = requests.post('https://digibody.avast.com/v1/web/leaks', json = email_d)
    list1 = avast.json()['value']
    for list_el in list1:
        postname.append(el)
        date.append(pd.to_datetime(list_el['leak_info']['date']))
        title.append(list_el['leak_info']['title'])
        description.append(list_el['leak_info']['description'])
    dict_={'postname':postname, 'date': date, 'title': title, 'description': description}
    time.sleep(0.2)
pd.DataFrame(dict_)


,postname,date,title,description
0,alekc@donet.ru,1970-01-01 00:25:13.900800,Combolist of 1.4 Billion Credentials,The proliferation of stolen or leaked database...
1,alekc@donet.ru,1970-01-01 00:25:49.411200,Collection #4 Combo List,"On January 7, 2019, an online user named Sanix..."
2,alekc@donet.ru,1970-01-01 00:25:07.507200,Exploit.in combolist,The proliferation of stolen or leaked database...
3,alekc@donet.ru,1970-01-01 00:24:53.856000,Active Account Cracking List,The proliferation of stolen or leaked-breach d...
4,alekc@donet.ru,1970-01-01 00:25:49.411200,Collection #5 Combo List,"On January 7, 2019, an online user named Sanix..."
5,alekc@donet.ru,1970-01-01 00:25:49.411200,2019 Antipublic Combo List,"On January 7, 2019, an online user named Sanix..."
6,alekc@donet.ru,1970-01-01 00:25:48.720000,Collection #2 Combo List,"On January 7, 2019, an online user named Sanix..."
7,alekc@donet.ru,1970-01-01 00:24:37.699200,VK,Popular Russian social networking platform VKo...
8,alekc@donet.ru,1970-01-01 00:25:48.374400,Collection #1 Combo List,"On January 7, 2019, an online user named Sanix..."
9,alekc@donet.ru,1970-01-01 00:26:15.504000,Sensitive Source,This source has been marked as sensitive due t...
